In [ ]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 KB 10.5 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow

In [ ]:
import os
from google.colab import drive
from PIL import Image
import numpy as np
import pickle
import keras_tuner
import tensorflow
from tensorflow import keras
from keras import regularizers
from keras_tuner import RandomSearch 
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras.preprocessing import image
from keras.callbacks import EarlyStopping

In [ ]:
# Authenticate and authorize Google Drive access
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# define path to seperated files (classes)
classes = '/content/drive/MyDrive/classes'
# if it doesn't already exist create one
if not os.path.exists(classes):
    os.makedirs(classes)

In [ ]:
val = '/content/drive/MyDrive/val'

In [ ]:
def normalizer(img):
  return img / 255.

In [ ]:
# ImageDataGenerator to load and normalize the images
datagen = ImageDataGenerator(preprocessing_function=normalizer)

In [ ]:
# create a generator to load and augment the images from the directory
train_generator = datagen.flow_from_directory(
    directory=classes,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

Found 51441 images belonging to 9 classes.


In [ ]:
input_shape = (200,200,3)
num_classes = 9 

In [ ]:
def build_model(hp):
    # input layer
    inputs = keras.Input(shape=input_shape)
    # flatten the input data
    flattened_input = layers.Flatten()(inputs)
    # number of layers to search
    num_layers = hp.Int('num_layers', min_value=1, max_value=5, step=1)
    # number of neurons for each layer to search
    units = [hp.Int('units_' + str(layer), min_value=32, max_value=512, step=32)
             for layer in range(num_layers)]
    # the activation function to search
    activation = hp.Choice('activation', ['relu', 'sigmoid', 'tanh'])
    # regularization to search
    regularization = hp.Choice('regularization', ['early_stop', 'drop_out', 'ridge'])
    # hidden layers
    if regularization == 'drop_out':
        for layer in range(num_layers):
            layer_output = layers.Dense(units[layer], activation=activation)(flattened_input if layer == 0 else layer_output)
            layer_output = layers.Dropout(hp.Float('dropout_rate_' + str(layer), min_value=0.0, max_value=0.5, step=0.1))(layer_output)
    else:
        for layer in range(num_layers):
            layer_output = layers.Dense(units[layer], activation=activation, kernel_regularizer=regularizers.l2(hp.Choice('l2_rate_' + str(layer), [0.01, 0.001, 0.0001])))(flattened_input if layer == 0 else layer_output)

    # output layer
    outputs = layers.Dense(num_classes, activation='softmax')(layer_output)

    # early stopping
    if regularization == 'early_stop':
        early_stop = EarlyStopping(monitor='val_loss', patience=5)
        callbacks = [early_stop]
    else:
        callbacks = []

    model = keras.Model(inputs=inputs, outputs=outputs)

    # compile the model
    model.compile(optimizer='adam',
                  loss=hp.Choice('loss_function', ['categorical_crossentropy', 'binary_crossentropy']),
                  metrics=['accuracy'])
  
    return model

In [ ]:
# hyperparameters 
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    directory='hyperparameters',
    project_name='multi-class-ffnn')

In [ ]:
val_generator = datagen.flow_from_directory(
    directory=val,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
)

Found 3985 images belonging to 9 classes.


In [ ]:
tuner.search(train_generator, epochs=32, validation_data=val_generator)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_layers
192               |192               |units_0
relu              |relu              |activation
early_stop        |early_stop        |regularization
0.001             |0.001             |l2_rate_0
binary_crossent...|binary_crossent...|loss_function
32                |32                |units_1
0.01              |0.01              |l2_rate_1

Epoch 1/32
1174/1608 [====================>.........] - ETA: 48:35 - loss: 0.8362 - accuracy: 0.1090

In [ ]:
# best hyperparameters and best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

# build the model with the best hyperparameters
model = build_model(best_hps)

In [ ]:
# compile the best model with the same optimizer, loss function, and metrics used during the hyperparameter search
best_model.compile(optimizer=best_hps.get('optimizer'),
                   loss=best_hps.get('loss'),
                   metrics=['accuracy'])

In [ ]:
# train model with train_generator
history = model.fit(train_generator, epochs=32, validation_data=val_generator)

In [ ]:
history

In [ ]:
test = '/content/drive/MyDrive/test'

In [ ]:
test_generator = datagen.flow_from_directory(
    directory=test,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical',
)

In [ ]:
# evaluation
test_loss, test_acc = model.evaluate(test_generator)

In [ ]:
test_acc